In [54]:
%pip install -Uq rich pandas numpy scipy 
%pip install -Uq matplotlib seaborn plotly 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3
[notice] To update, run: C:\Users\Илья\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3
[notice] To update, run: C:\Users\Илья\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [55]:
from scipy import stats as st
import math

from rich import print
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [56]:
rides = pd.read_csv('rides_go.csv')
subscriptions = pd.read_csv('subscriptions_go.csv')
users = pd.read_csv('users_go.csv')

In [57]:
rides['date'] = pd.to_datetime(rides['date'])
rides.head()

,user_id,distance,duration,date
0,1,4409.919140,25.599769,2021-01-01
1,1,2617.592153,15.816871,2021-01-18
2,1,754.159807,6.232113,2021-04-20
3,1,2694.783254,18.511000,2021-08-11
4,1,4028.687306,26.265803,2021-08-28


In [58]:
data = users.merge(rides, on='user_id')
data['date_month'] = data['date'].dt.month
data.head()

,user_id,name,age,city,subscription_type,distance,duration,date,date_month
0,1,Кира,22,Тюмень,ultra,4409.919140,25.599769,2021-01-01,1
1,1,Кира,22,Тюмень,ultra,2617.592153,15.816871,2021-01-18,1
2,1,Кира,22,Тюмень,ultra,754.159807,6.232113,2021-04-20,4
3,1,Кира,22,Тюмень,ultra,2694.783254,18.511000,2021-08-11,8
4,1,Кира,22,Тюмень,ultra,4028.687306,26.265803,2021-08-28,8


In [59]:
data_free = data.query('subscription_type == "free"')
data_ultra = data.query('subscription_type == "ultra"')
months_hot = [1, 2, 3, 10, 11, 12]
months_cold = [4, 5, 6, 7, 8, 9]
display(data_free.head())
data_ultra.head()

,user_id,name,age,city,subscription_type,distance,duration,date,date_month
6672,700,Айдар,22,Омск,free,2515.690719,14.944286,2021-01-02,1
6673,700,Айдар,22,Омск,free,846.932642,16.234663,2021-02-01,2
6674,700,Айдар,22,Омск,free,4004.434142,20.016628,2021-02-04,2
6675,700,Айдар,22,Омск,free,1205.911290,9.782872,2021-02-10,2
6676,700,Айдар,22,Омск,free,3047.379435,17.427673,2021-02-14,2


,user_id,name,age,city,subscription_type,distance,duration,date,date_month
0,1,Кира,22,Тюмень,ultra,4409.919140,25.599769,2021-01-01,1
1,1,Кира,22,Тюмень,ultra,2617.592153,15.816871,2021-01-18,1
2,1,Кира,22,Тюмень,ultra,754.159807,6.232113,2021-04-20,4
3,1,Кира,22,Тюмень,ultra,2694.783254,18.511000,2021-08-11,8
4,1,Кира,22,Тюмень,ultra,4028.687306,26.265803,2021-08-28,8


In [60]:
data_cd = data
data_cd.duration = data_cd.duration.apply(np.ceil)
users_pivot = pd.pivot_table(
    data_cd, 
    index=['user_id', 'date_month'],
    #columns=['subscription_type'],
    values=['distance', 'duration'], 
    aggfunc={
        'distance': 'sum',
        'user_id': 'count',
        'duration': 'sum',
    }
)
users_pivot.columns = ['month_distance', 'month_duration', 'month_rides']
users_pivot['subscription_type'] = \
    users_pivot.apply(lambda x: users[users.user_id == x.name[0]].iloc[0][-1], axis=1)
users_pivot.head(10)

C:\Users\Илья\AppData\Local\Temp\ipykernel_12840\2124977302.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  users_pivot.apply(lambda x: users[users.user_id == x.name[0]].iloc[0][-1], axis=1)


month_distance  month_duration  month_rides  \
user_id date_month                                                
1       1              7027.511294            42.0            2   
        4               754.159807             7.0            1   
        8              6723.470560            46.0            2   
        10             5809.911100            32.0            2   
        11             7003.499363            56.0            3   
        12             6751.629942            28.0            2   
2       3             10187.723006            63.0            3   
        4              6164.381824            40.0            2   
        6              3255.338202            14.0            1   
        7              6780.722964            48.0            2   

                   subscription_type  
user_id date_month                    
1       1                      ultra  
        4                      ultra  
        8                      ultra  
        10                     ultra  
        11                     ultra  
        12                     ultra  
2       3                      ultra  
        4                      ultra  
        6                      ultra  
        7                      ultra

In [61]:
import warnings

def calculate_month_income(series: pd.Series) -> int:
    user_id: int = series.name[0]
    duration: int = np.ceil(series.month_duration)
    rides_count: int = series.month_rides
    subscription_type: str = series.subscription_type

    type_prices = subscriptions[subscriptions.subscription_type == subscription_type]
    
    # тут использую инты для ускорения работы, так как тут всё равно неоткуда появиться
    # дробям в нынешней итерации - всё слишком округлённое по заданию
    # но в идеале бы всё во float
    total_income: int = int(type_prices.subscription_fee)
    total_income += int(rides_count * type_prices.start_ride_price)
    total_income += int(duration * type_prices.minute_price)
    return total_income
    

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    users_pivot['month_income'] = users_pivot.apply(calculate_month_income, axis=1)
users_pivot.head(10)

month_distance  month_duration  month_rides  \
user_id date_month                                                
1       1              7027.511294            42.0            2   
        4               754.159807             7.0            1   
        8              6723.470560            46.0            2   
        10             5809.911100            32.0            2   
        11             7003.499363            56.0            3   
        12             6751.629942            28.0            2   
2       3             10187.723006            63.0            3   
        4              6164.381824            40.0            2   
        6              3255.338202            14.0            1   
        7              6780.722964            48.0            2   

                   subscription_type  month_income  
user_id date_month                                  
1       1                      ultra           451  
        4                      ultra           241  
        8                      ultra           475  
        10                     ultra           391  
        11                     ultra           535  
        12                     ultra           367  
2       3                      ultra           577  
        4                      ultra           439  
        6                      ultra           283  
        7                      ultra           487

In [62]:
data_free = data.query('subscription_type == "free"')
data_ultra = data.query('subscription_type == "ultra"')
display(data_free.head())
data_ultra.head()

,user_id,name,age,city,subscription_type,distance,duration,date,date_month
6672,700,Айдар,22,Омск,free,2515.690719,15.0,2021-01-02,1
6673,700,Айдар,22,Омск,free,846.932642,17.0,2021-02-01,2
6674,700,Айдар,22,Омск,free,4004.434142,21.0,2021-02-04,2
6675,700,Айдар,22,Омск,free,1205.911290,10.0,2021-02-10,2
6676,700,Айдар,22,Омск,free,3047.379435,18.0,2021-02-14,2


,user_id,name,age,city,subscription_type,distance,duration,date,date_month
0,1,Кира,22,Тюмень,ultra,4409.919140,26.0,2021-01-01,1
1,1,Кира,22,Тюмень,ultra,2617.592153,16.0,2021-01-18,1
2,1,Кира,22,Тюмень,ultra,754.159807,7.0,2021-04-20,4
3,1,Кира,22,Тюмень,ultra,2694.783254,19.0,2021-08-11,8
4,1,Кира,22,Тюмень,ultra,4028.687306,27.0,2021-08-28,8


## Будет ли помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки?

H0: Помесячная выручка от пользователей с подпиской будет равна выручке пользователей без подписки
H1: Помесячная выручка от пользователей с подпиской будет больше выручки пользователей без подписки

In [63]:
alpha = 0.05

results = st.ttest_ind (
    users_pivot.query('subscription_type == "ultra"').month_income,
    users_pivot.query('subscription_type == "free"').month_income,
    alternative='two-sided'
)


print(f'p-value: {results.pvalue}')

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

p-value: 8.454877789082266e-29

Отвергаем нулевую гипотезу

## Среднее количество поездок в месяц пользователей с подпиской больше, чем у пользователей без подписки?

H0: Среднее количество поездок в месяц пользователей с подпиской столько же, сколько у пользователей без подписки
H1: Среднее количество поездок в месяц пользователей с подпиской больше, чем у пользователей без подписки

In [64]:
alpha = 0.05

results = st.ttest_ind (
    users_pivot.query('subscription_type == "ultra"').month_rides,
    users_pivot.query('subscription_type == "free"').month_rides,
    alternative='two-sided'
)


print(f'p-value: {results.pvalue}')

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

p-value: 1.4510810862270678e-51

Отвергаем нулевую гипотезу

## Средняя дистанция поездки "холодные" месяца (с октября по март) отличается от "тёплых"?

H0: Средняя дистанция поездки "холодных" месяцев такая же как дистанция "тёплых" месяцев
H1: Средняя дистанция поездки "холодных" месяцев отличается от дистанции "тёплых" месяцев

In [65]:
alpha = 0.05

results = st.ttest_ind (
    users_pivot.query('date_month == @months_hot').month_distance,
    users_pivot.query('date_month == @months_cold').month_distance,
    alternative='two-sided'
)

print(f'p-value: {results.pvalue}')

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

p-value: 0.19365908135594664

Нет оснований отвергнуть нулевую гипотезу

## Среднее время езды на самокате - 30 минут?

H0: Среднее время езды на самокате - 30 минут
H1: Среднее время езды на самокате - 30 минут

In [66]:
alpha = 0.05
target_value = 30

results = st.ttest_1samp(
    data.duration,
    target_value,
    alternative='two-sided'
)

print(f'p-value: {results.pvalue}')

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

p-value: 0.0

Отвергаем нулевую гипотезу